<a href="https://colab.research.google.com/github/JamieCDev20/GAM705/blob/main/705Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setup

Imports

In [ ]:
!pip install pyquaternion
!pip install PyDrive

In [28]:
import numpy as np
from pyquaternion import Quaternion
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from random import random as r

In [17]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

File Reader

In [ ]:
class AnimFileReader():
  ## "#" breaks between nodes | ":" breaks between values

  def __init__(self, _path):
    self.path = _path
    self.animData = np.array(float)
    self.__ReadAnim__(self.path)

  def __ReadAnim__(self, path):
    file = open(path, 'r')
    overallData = file.read()
    overallData = overallData.split("SS")
    
    del overallData[0]

    jointCount = len(overallData)
    frameCount = len(overallData[0].split('#')[0].split(':')) - 1
    
    dataLayer = np.array(float)

    trainingData = np.array(float)

    for j in range(0, frameCount):
      for i in range(0, jointCount):

        rots = overallData[i].split('#')[0].split(':')
        poss = overallData[i].split('#')[1].split(':')

        del rots[0]
        del poss[0]

        #print(len(rots))
        #print(rots[0], rots[1], rots[2], rots[3])
        #print(dataLayer.shape)
        temp = rots[j].split(',')
        dataLayer = np.append(dataLayer, temp)
        
        temp = poss[j].split(',')
        dataLayer = np.append(dataLayer, temp)
        #print(dataLayer.shape)
      dataLayer = np.delete(dataLayer, 0)
      trainingData = np.append(trainingData, dataLayer);
      dataLayer = np.array(float)
      print("Completed: ", j + 1)

    print("Total its: ", (i + 1) * (j + 1))
    
    trainingData = np.delete(trainingData, 0)
    
    print(trainingData.shape)
    print(trainingData[0])
    
    self.animData = trainingData;
   

#File Read

In [ ]:
fr = AnimFileReader("SHO_EXORCISMWALK_1_UNCLEAN.txt")

In [ ]:
data = fr.animData
print(fr.animData.shape)
print(data.shape)
data = np.reshape(data, [221, 168])
print(fr.animData.shape)
print(data.shape)

(37128,)
(37128,)
(37128,)
(221, 168)


In [ ]:
network = Sequential(
    Input()
)

Testing

In [ ]:
a = np.array([1, 2, 3, 4, 5], float)
b = np.array([6, 6, 6, 6, 6], float)
c = b-a
b = np.append(b, c);

print(b)


[6. 6. 6. 6. 6. 5. 4. 3. 2. 1.]


#GAN Training

##Networks

Discriminator

In [23]:
discriminator = Sequential([
    Input(shape=(1)),
    Dense(5, activation='relu'),
    Dense(1)
])
discriminator.compile(optimizer="Adam", metrics=["Accuracy"])

Generator

In [24]:
generator = Sequential([
      Input(shape=(1)),
      Dense(5, activation='relu'),
      Dense(1)
])
generator.compile(optimizer="Adam", metrics=["Accuracy"])

##Training Structure

Training Vars

In [10]:
batchSize = 64
iterations = 100000
fakeDistribution = 0.5

In [45]:
def Func(x):
  return [[0.08 * (x[0]-25)**2]]

In [47]:
def GenerateBatch():
  batchDistribution = np.random.rand(batchSize)
  batchDistribution = [(0, 1)[int(x > fakeDistribution)] for x in batchDistribution]
  xTrain = [[generator.predict, Func][x] for x in batchDistribution]
  return xTrain, batchDistribution

a = [generator.predict, Func][0]
print(a([12]))

[[-13.084098]]


Training

In [34]:
print(GenerateBatch())
#Do the training loop here

([<bound method Model.predict of <keras.engine.sequential.Sequential object at 0x7f99c88fc1d0>>, <bound method Model.predict of <keras.engine.sequential.Sequential object at 0x7f99c88fc1d0>>, <function Func at 0x7f99c88e98c0>, <function Func at 0x7f99c88e98c0>, <bound method Model.predict of <keras.engine.sequential.Sequential object at 0x7f99c88fc1d0>>, <function Func at 0x7f99c88e98c0>, <bound method Model.predict of <keras.engine.sequential.Sequential object at 0x7f99c88fc1d0>>, <bound method Model.predict of <keras.engine.sequential.Sequential object at 0x7f99c88fc1d0>>, <bound method Model.predict of <keras.engine.sequential.Sequential object at 0x7f99c88fc1d0>>, <function Func at 0x7f99c88e98c0>, <bound method Model.predict of <keras.engine.sequential.Sequential object at 0x7f99c88fc1d0>>, <function Func at 0x7f99c88e98c0>, <bound method Model.predict of <keras.engine.sequential.Sequential object at 0x7f99c88fc1d0>>, <function Func at 0x7f99c88e98c0>, <bound method Model.predict 